# Distribución del Repositorio

```
|- app
|  |- model
|  |  |- Dockerfile
|  |  |- Main.py
|  |  |- requirements.txt
|  |
|  |- ui
|     |- Dockerfile
|     |- Main.py
|     |- requirements.txt
|  
|  # Contiene la aplicación que ejecuta el modelo en contenedores.
|  # - `model`: Backend de la aplicación.
|  # - `ui`: Frontend de la aplicación.

|- docker-compose
|  # Archivo para orquestar los contenedores.

|- data
|  |- clean_data
|  |  # CSV post limpieza y preparado para el modelado.
|  |
|  |- raw_data
|  |  # Datos originales en formato JSON.
|  |
|  |- transformed_data
|     # Datos estructurados según los requisitos del modelo.

|- notebooks
|  |- DataWrangling.ipynb
|  |  # Limpieza y preprocesamiento de datos.
|  |
|  |- EDA.ipynb
|  |  # Análisis descriptivo de los datos.
|  |
|  |- Experiments.ipynb
|  |  # Pruebas de modelos y selección del más adecuado.
|  |
|  |- Pipeline.py
|     # Flujo de Prefect para registrar y reentrenar modelos cuando sea necesario.
```

<h1 style="color: #2E86C1; text-align: center;">Data Wrangling</h1>

<p style="color: #34495E; font-size: 16px; line-height: 1.6;">
    <strong>Comencé este examen</strong> siguiendo los pasos para la lectura y preprocesado inicial del dataset: 
    tomé el <code>Json</code>, lo normalicé, recorté el dataset a las variables de importancia, las renombré, 
    hice el <i>feature engineering</i> para la variable de respuesta y comencé a ir y venir al EDA para obtener insights.
</p>

<p style="color: #2874A6; font-size: 16px; line-height: 1.6;">
    Primero hice limpieza clásica de textos:
    <ul>
        <li>Eliminé todo lo que no fuera alfanumérico.</li>
        <li>Estandaricé a minúsculas.</li>
        <li>Expandí contracciones.</li>
        <li>Quité <i>stopwords</i>.</li>
    </ul>
</p>

<p style="color: #148F77; font-size: 16px; line-height: 1.6;">
    Después, cuando comencé a checar el EDA, noté que había palabras que eran casi constantes a lo largo de todas las quejas.
    Así que decidí generar una función que recortara aquellas palabras que aparecieran en cierta cantidad de documentos, 
    y finalmente la dejé para el <b>80% de los documentos</b>.
</p>

<p style="color: #D68910; font-size: 16px; line-height: 1.6;">
    Casi para terminar, noté que había categorías con muy pocos registros, 
    así que tomé un enfoque similar al de las palabras muy repetidas y eliminé todas aquellas categorías con un conteo de registros menor a <b>10</b>.
</p>

<p style="color: #76448A; font-size: 16px; line-height: 1.6;">
    Por último, apliqué ambos métodos en <i>batch processing</i> y guardé un <b>save state</b> del dataset en <code>transformed data</code>.
</p>


<h1 style="color: #2E86C1; text-align: center;">EDA</h1>

<p style="color: #34495E; font-size: 16px; line-height: 1.6;">
    El <b>EDA</b> fue mi guía para el <i>data wrangling</i>, ya que los insights que iba obteniendo me servían para la limpieza. 
    Entre lo más relevante está:
    <ul>
        <li>Notar las clases con pocos registros.</li>
        <li>Identificar muchas <i>stopwords</i>.</li>
        <li>Detectar palabras muy repetidas.</li>
    </ul>
    Fue un proceso iterativo, pero finalmente dejé limpio el dataset y procedí a la visualización.
</p>

<p style="color: #2874A6; font-size: 16px; line-height: 1.6;">
    Para ello reutilicé una función que ya había hecho hace tiempo para graficar un <b>wordcloud</b> por cada categoría junto a una colección del <b>top 10</b>:
    <ol>
        <li>Unigramas.</li>
        <li>Bigramas.</li>
        <li>Trigramas.</li>
    </ol>
    Esto permitió evaluar qué tan distintas eran las categorías después de la limpieza, y los resultados fueron satisfactorios.
</p>


<h1 style="color: #2E86C1; text-align: center;">Experimentación</h1>

<p style="color: #34495E; font-size: 16px; line-height: 1.6;">
    Para el modelado, comencé siguiendo lo que aprendí en minería de textos. Pensaba en utilizar 
    <b>regresión logística</b> y <b>random forest</b>, porque ambos me dieron muy buenos resultados en esa clase.
</p>

<p style="color: #2874A6; font-size: 16px; line-height: 1.6;">
    Utilicé <b>TF-IDF</b> (<i>Term Frequency-Inverse Document Frequency</i>) para vectorizar los textos, ya que va muy bien con 
    el tipo de limpieza que había utilizado.
</p>

<p style="color: #148F77; font-size: 16px; line-height: 1.6;">
    En la primera iteración del modelado me fue bastante bien, pero tuve un problema con el encodeado de las etiquetas y 
    con la vectorización del texto. A la hora de pasar un nuevo <i>input</i>, no estaba codificado en la matriz de frecuencia directa e inversa.
</p>

<p style="color: #D68910; font-size: 16px; line-height: 1.6;">
    Así que tuve que rehacer el modelado, definiendo un <b>pipeline</b> que implementara el vectorizador y, posteriormente, 
    lo loggeara como artefacto para cuando necesitara predecir con el modelo desde <b>MLflow</b>.
</p>

<p style="color: #76448A; font-size: 16px; line-height: 1.6;">
    Estuve jugando mucho con los <i>grid search</i> y mis modelos, pero finalmente decidí quedarme con:
    <ul>
        <li>Penalización <b>L2</b>.</li>
        <li>Un solver que combinara con esta penalización.</li>
        <li>Un <i>cross-validation</i> de <b>5</b>.</li>
        <li><b>Verbose</b> de 1 y 2 (aunque este último no tuvo impacto contrario a lo que esperaba).</li>
    </ul>
</p>


<h1 style="color: #2E86C1; text-align: center;">Pipeline</h1>

<p style="color: #34495E; font-size: 16px; line-height: 1.6;">
    Para el <b>pipeline</b>, simplemente desmenucé lo que ya tenía en la experimentación, pero solo implementando 
    la <b>regresión logística</b>.
</p>

<p style="color: #2874A6; font-size: 16px; line-height: 1.6;">
    Además, añadí la tarea <code>SetChamp</code> para tomar el resultado de la función <code>getBestModel</code> y 
    convertirlo en <b>champion</b>. También cambié la etiqueta por <code>champ</code> para que fuera más fácil identificarla entre todos los modelos.
</p>

<p style="color: #148F77; font-size: 16px; line-height: 1.6;">
    Corrí el flujo y todo funcionó <b>perfecto</b>.
</p>



<h1 style="color: #2E86C1; text-align: center;">App</h1>

<p style="color: #34495E; font-size: 16px; line-height: 1.6;">
    Finalmente, para el desarrollo de la <b>app</b>, tomé lo que ya habíamos generado para <i>nyc-taxi-model</i> e hice unos cuantos ajustes en el <code>yaml</code> y los <code>.py</code>.
</p>

<p style="color: #2874A6; font-size: 16px; line-height: 1.6;">
    Esta parte me generó algo de conflicto, especialmente porque, siguiendo la lógica de <i>taxi model</i>, enviaba un diccionario, 
    y tuve problemas para extraerlo y pasárselo al modelo aunque solo tuviera un valor.
</p>

<p style="color: #148F77; font-size: 16px; line-height: 1.6;">
    Estaba considerando usar un <code>zip</code>, pero luego me di cuenta de que, al ser solo una variable predictora, 
    podía enviarla directamente al <b>back</b>. Así lo implementé, hice pruebas, y <b>funciona perfecto</b>.
</p>
